In [1]:
import pandas as pd
df = pd.read_csv('D:/Python_WC/Final_project/Multi-Task_News_Intelligence_System/Data/news.tsv', sep='\t',on_bad_lines='skip')
df.head()


,News ID,Category,Topic,Headline,News body,Title entity,Entity content
0,N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI...","{""Atlanta United's"": 'Atlanta United FC'}","{'Atlanta United FC': {'type': 'item', 'id': '..."
1,N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...,"{'DC': 'Washington, D.C.'}","{'Washington, D.C.': {'type': 'item', 'id': 'Q..."
2,N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...,{},{}
3,N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...,{},{}
4,N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec...",{},{}


In [2]:
df.columns

Index(['News ID', 'Category', 'Topic', 'Headline', 'News body', 'Title entity',
       'Entity content'],
      dtype='object')

In [3]:
df.count()

News ID           113762
Category          113762
Topic             113762
Headline          113762
News body         113704
Title entity      113762
Entity content    113762
dtype: int64

In [4]:
df.value_counts

<bound method DataFrame.value_counts of         News ID Category                  Topic  \
0        N10000   sports                 soccer   
1        N10001     news           newspolitics   
2        N10002     news                 newsus   
3        N10003     news           newspolitics   
4        N10004     news              newsworld   
...         ...      ...                    ...   
113757  N123757   sports        soccer_fifa_wwc   
113758  N123758   sports           baseball_mlb   
113759  N123759   sports  basketball_nba_videos   
113760  N123760     news           newspolitics   
113761  N123761   sports           football_nfl   

                                                 Headline  \
0       Predicting Atlanta United's lineup against Col...   
1       Mitch McConnell: DC statehood push is 'full bo...   
2                 Home In North Highlands Damaged By Fire   
3       Meghan McCain blames 'liberal media' and 'thir...   
4                                 Today in

In [5]:
import ast
import re
import spacy

nlp = spacy.load("en_core_web_sm")

df["Headline"] = df["Headline"].astype(str)
df["Title entity"] = df["Title entity"].astype(str)

COUNTRIES = ["United States", "India", "Brazil", "China", "Mexico", "Canada"]
PERSON_PATTERN = r"^[A-Z][a-z]+(\s[A-Z][a-z]+)+$"
ORG_KEYWORDS = ["Corporation", "Authority", "Committee", "Association", "University", "Agency", "Company", "FC", "Ltd"]


def infer_entity_type(expanded):
    expanded = expanded.strip()

    if re.match(PERSON_PATTERN, expanded):
        return "PERSON"

    if expanded in COUNTRIES:
        return "LOCATION"

    if any(k in expanded for k in ORG_KEYWORDS):
        return "ORG"

    return "MISC"

def convert_to_bio(text, entity_string):
    tokens = text.split()
    tags = ["O"] * len(tokens)

    if entity_string == "{}":
        return tokens, tags

    try:
        ent_dict = ast.literal_eval(entity_string)
    except:
        return tokens, tags

    lower_tokens = [w.lower().strip(".,!?") for w in tokens]

    for surface, expanded in ent_dict.items():
        clean_surface = surface.replace("'s", "").strip()
        stoks = clean_surface.split()
        stoks = [w.lower().strip(".,!?") for w in stoks]
        n = len(stoks)

        ent_type = infer_entity_type(expanded)

        # Search entity span safely
        for i in range(len(tokens)):
            try:
                if lower_tokens[i:i+n] == stoks:
                    tags[i] = f"B-{ent_type}"
                    for j in range(i+1, i+n):
                        if j < len(tags):   # SAFETY CHECK
                            tags[j] = f"I-{ent_type}"
            except:
                continue

    return tokens, tags



sentences = []
labels = []

for _, row in df.iterrows():
    s, t = convert_to_bio(row["Headline"], row["Title entity"])
    sentences.append(s)
    labels.append(t)

print("DATA READY — Samples:", len(sentences))

DATA READY — Samples: 113762


In [14]:
# ENCODING & VOCAB GENERATION

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

word_set = set(w for s in sentences for w in s)
tag_set = set(t for seq in labels for t in seq)

# Build word2idx
word2idx = {w: i+2 for i, w in enumerate(sorted(word_set))}
word2idx["<PAD>"] = 0
word2idx["<UNK>"] = 1

# Build tag2idx
tag2idx = {t: i for i, t in enumerate(sorted(tag_set))}
idx2tag = {v: k for k, v in tag2idx.items()}

MAX_LEN = 60

# Encode words
X = [[word2idx.get(w, 1) for w in seq] for seq in sentences]
X = pad_sequences(X, maxlen=MAX_LEN, padding="post")

# Encode tags
y = [[tag2idx[t] for t in seq] for seq in labels]
y = pad_sequences(y, maxlen=MAX_LEN, padding="post")
y_cat = to_categorical(y, num_classes=len(tag2idx))

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y_cat, test_size=0.2, random_state=42
)

# 🔥 FIX: VOCAB_SIZE must match the highest index in word2idx
VOCAB_SIZE = max(word2idx.values()) + 1
NUM_TAGS = len(tag2idx)

print("VOCAB SIZE:", VOCAB_SIZE)
print("NUM TAGS:", NUM_TAGS)


VOCAB SIZE: 104705
NUM TAGS: 8


In [9]:
!pip install gensim


[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# BUILD EMBEDDINGS
# GloVe

GLOVE_PATH = "D:/Python_WC/Final_project/glove.6B/glove.6B.100d.txt"
import numpy as np

# Load GloVe vectors
glove_index = {}
with open(GLOVE_PATH, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        glove_index[word] = vector

print("Total embeddings found:", len(glove_index))

# IMPORTANT: VOCAB_SIZE must already be set as:
# VOCAB_SIZE = max(word2idx.values()) + 1

# Build embedding matrix with correct size
embedding_glove = np.zeros((VOCAB_SIZE, 100))

# Fill matrix
for w, i in word2idx.items():
    embedding_glove[i] = glove_index.get(
        w,
        np.random.normal(scale=0.6, size=(100,))
    )


Total embeddings found: 400000


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)

# ============================================
# BiLSTM Model (Corrected)
# ============================================

def build_bilstm(embedding_matrix):
    inp = Input(shape=(MAX_LEN,))

    # IMPORTANT:
    # VOCAB_SIZE must be: max(word2idx.values()) + 1
    # embedding_matrix must have shape (VOCAB_SIZE, 100)

    emb = Embedding(
        input_dim=VOCAB_SIZE,     # <-- matches your corrected VOCAB_SIZE
        output_dim=100,
        weights=[embedding_matrix],
        mask_zero=True,
        trainable=False
    )(inp)

    x = Bidirectional(LSTM(128, return_sequences=True))(emb)
    out = TimeDistributed(Dense(NUM_TAGS, activation="softmax"))(x)

    model = Model(inp, out)
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [17]:
results = []
models_dict = {}
print("\nTraining MODEL-2: BiLSTM + GloVe")
bilstm_model = build_bilstm(embedding_glove)
bilstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, callbacks=[early_stop])
models_dict["1"] = bilstm_model



Training MODEL-2: BiLSTM + GloVe
Epoch 1/5
2845/2845 ━━━━━━━━━━━━━━━━━━━━ 118s 41ms/step - accuracy: 0.9161 - loss: 0.3088 - val_accuracy: 0.9264 - val_loss: 0.2450
Epoch 2/5
2845/2845 ━━━━━━━━━━━━━━━━━━━━ 108s 38ms/step - accuracy: 0.9346 - loss: 0.2132 - val_accuracy: 0.9367 - val_loss: 0.2083
Epoch 3/5
2845/2845 ━━━━━━━━━━━━━━━━━━━━ 108s 38ms/step - accuracy: 0.9437 - loss: 0.1775 - val_accuracy: 0.9396 - val_loss: 0.1985
Epoch 4/5
2845/2845 ━━━━━━━━━━━━━━━━━━━━ 108s 38ms/step - accuracy: 0.9508 - loss: 0.1529 - val_accuracy: 0.9409 - val_loss: 0.1922
Epoch 5/5
2845/2845 ━━━━━━━━━━━━━━━━━━━━ 111s 39ms/step - accuracy: 0.9564 - loss: 0.1329 - val_accuracy: 0.9422 - val_loss: 0.1993


In [19]:
from sklearn.metrics import classification_report

def evaluate(model, model_name, embed_name):
    preds = model.predict(X_val).argmax(axis=-1)
    true_tags = y_val.argmax(axis=-1)

    y_true = []
    y_pred = []

    for t_seq, p_seq in zip(true_tags, preds):
        for t, p in zip(t_seq, p_seq):
            if idx2tag[t] != "O":
                y_true.append(idx2tag[t])
                y_pred.append(idx2tag[p])

    report = classification_report(
        y_true,
        y_pred,
        output_dict=True,
        zero_division=0
    )

    weighted = report["weighted avg"]

    return {
        "Model": model_name,
        "Embedding": embed_name,
        "Precision": weighted["precision"],
        "Recall": weighted["recall"],
        "F1 Score": weighted["f1-score"]
    }


results.append(evaluate(bilstm_model, "BiLSTM", "GloVe"))


df_result = pd.DataFrame(results)
df_result.to_csv("D:/Python_WC/Final_project/Multi-Task_News_Intelligence_System/NER/model_comparison_NER.csv", index=False)
print("Evaluation Completed and File Saved")

712/712 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step
Evaluation Completed and File Saved


In [24]:
best_row = df_result.loc[df_result["F1 Score"].idxmax()]
print("\nBEST MODEL SELECTED:\n", best_row)

best_tuple = (best_row["Model"], best_row["Embedding"])

best_model = {
    
    ("BiLSTM", "GloVe"): bilstm_model,
    
}[best_tuple]




BEST MODEL SELECTED:
 Model          BiLSTM
Embedding       GloVe
Precision    0.922337
Recall       0.010013
F1 Score     0.012809
Name: 0, dtype: object


In [26]:
best_model.save("D:/Python_WC/Final_project/Multi-Task_News_Intelligence_System/NER/final_best_model.keras")
print("\nSAVED BEST MODEL!")


SAVED BEST MODEL!


In [20]:
import pickle

with open("D:/Python_WC/Final_project/Multi-Task_News_Intelligence_System/NER/word2idx.pkl", "wb") as f:
    pickle.dump(word2idx, f)

In [21]:
with open("D:/Python_WC/Final_project/Multi-Task_News_Intelligence_System/NER/idx2tag.pkl", "wb") as f:
    pickle.dump(idx2tag, f)

In [22]:
with open("D:/Python_WC/Final_project/Multi-Task_News_Intelligence_System/NER/max_len.txt", "w") as f:
    f.write(str(MAX_LEN))
